### Setup

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Setup imports
from crewai import Agent, Task, Crew, LLM
import os
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv()

# get default OpenRouter API key
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")
else:
    print("Using OpenRouter API key from environment variable.")


Using OpenRouter API key from environment variable.


In [3]:
# Initialize the LLM
llm = LLM(
    model="openrouter/openai/gpt-5-mini",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [4]:
# setup chroma DB + Client
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

# Set custom storage location
os.environ["CREWAI_STORAGE_DIR"] = "./chroma_db"

### Agents

In [5]:
# writing and reviewing agents
content_writer = Agent(
    role="Educational Content Writer",
    goal="Create engaging, informative content that thoroughly explains the assigned topic and provides valuable insights to the reader",
    backstory=  "You are a talented educational writer with expertise in creating clear, engaging "
                "content. You have a gift for explaining complex concepts in accessible language "
                "and organizing information in a way that helps readers build their understanding.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

content_reviewer = Agent(
    role="Educational Content Reviewer and Editor",
    goal="Ensure content is accurate, comprehensive, well-structured, and maintains consistency with previously written sections",
    backstory=  "You are a meticulous editor with years of experience reviewing educational"
                "content. You have an eye for detail, clarity, and coherence. You excel at"
                "improving content while maintaining the original author's voice and ensuring"
                "consistent quality across multiple sections.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Tasks

In [6]:
write_section_task = Task(
    description=(
        "Write a comprehensive section on the topic: {section_title}\n\n"
        "Section description: {section_description}\n"
        "Target audience: {audience_level} level learners\n\n"
        "Your content should:\n"
        "1. Begin with a brief introduction to the section topic\n"
        "2. Explain all key concepts clearly with examples\n"
        "3. Include practical applications or exercises where appropriate\n"
        "4. End with a summary of key points\n"
        "5. Be approximately 500-800 words in length\n\n"
        "Format your content in Markdown with appropriate headings, lists, and emphasis.\n\n"
        "Previously written sections:\n"
        "{previous_sections}\n\n"
        "Make sure your content maintains consistency with previously written sections "
        "and builds upon concepts that have already been explained."
    ),
    expected_output=(
        "A well-structured, comprehensive section in Markdown format that thoroughly "
        "explains the topic and is appropriate for the target audience."
    ),
    agent=content_writer
)

review_section_task = Task(
    description=(
        "Review and improve the following section on \"{section_title}\":\n\n"
        "{draft_content}\n\n"
        "Target audience: {audience_level} level learners\n\n"
        "Previously written sections:\n"
        "{previous_sections}\n\n"
        "Your review should:\n"
        "1. Fix any grammatical or spelling errors\n"
        "2. Improve clarity and readability\n"
        "3. Ensure content is comprehensive and accurate\n"
        "4. Verify consistency with previously written sections\n"
        "5. Enhance the structure and flow\n"
        "6. Add any missing key information\n\n"
        "Provide the improved version of the section in Markdown format."
    ),
    expected_output=(
        "An improved, polished version of the section that maintains the original "
        "structure but enhances clarity, accuracy, and consistency."
    ),
    agent=content_reviewer,
    context=[write_section_task]
)

### Create the crew

In [7]:
# Create the crew with agents and tasks
content_crew = Crew(
    agents=[content_writer, content_reviewer],
    tasks=[write_section_task, review_section_task],
    verbose=True,
    # memory=True,
)

### Create the flow

In [8]:
import json
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from crewai import LLM
from crewai.flow.flow import Flow, listen, start


# Define our models for structured data
class Section(BaseModel):
    title: str = Field(description="Title of the section")
    description: str = Field(description="Brief description of what the section should cover")

class GuideOutline(BaseModel):
    title: str = Field(description="Title of the guide")
    introduction: str = Field(description="Introduction to the topic")
    target_audience: str = Field(description="Description of the target audience")
    sections: List[Section] = Field(description="List of sections in the guide")
    conclusion: str = Field(description="Conclusion or summary of the guide")

# Define our flow state
class GuideCreatorState(BaseModel):
    topic: str = ""
    audience_level: str = ""
    guide_outline: GuideOutline = None
    sections_content: Dict[str, str] = {}

class GuideCreatorFlow(Flow[GuideCreatorState]):
    """Flow for creating a comprehensive guide on any topic"""

    @start()
    def get_user_input(self):
        """Get input from the user about the guide topic and audience"""
        print("\n=== Create Your Comprehensive Guide ===\n")

        # Get user input
        self.state.topic = input("What topic would you like to create a guide for? ")

        # Get audience level with validation
        while True:
            audience = input("Who is your target audience? (beginner/intermediate/advanced) ").lower()
            if audience in ["beginner", "intermediate", "advanced"]:
                self.state.audience_level = audience
                break
            print("Please enter 'beginner', 'intermediate', or 'advanced'")

        print(f"\nCreating a guide on {self.state.topic} for {self.state.audience_level} audience...\n")
        return self.state

    @listen(get_user_input)
    def create_guide_outline(self, state):
        """Create a structured outline for the guide using a direct LLM call"""
        print("Creating guide outline...")

        # Initialize the LLM
        llm.response_format=GuideOutline

        # Create the messages for the outline
        messages = [
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f"""
            Create a detailed outline for a comprehensive guide on "{state.topic}" for {state.audience_level} level learners.

            The outline should include:
            1. A compelling title for the guide
            2. An introduction to the topic
            3. 4-6 main sections that cover the most important aspects of the topic
            4. A conclusion or summary

            For each section, provide a clear title and a brief description of what it should cover.
            """}
        ]

        # Make the LLM call with JSON response format
        response = llm.call(messages=messages)

        # Parse the JSON response
        outline_dict = json.loads(response)
        self.state.guide_outline = GuideOutline(**outline_dict)

        # Ensure output directory exists before saving
        os.makedirs("output", exist_ok=True)

        # Save the outline to a file
        with open("output/guide_outline.json", "w") as f:
            json.dump(outline_dict, f, indent=2)

        print(f"Guide outline created with {len(self.state.guide_outline.sections)} sections")
        return self.state.guide_outline

    @listen(create_guide_outline)
    def write_and_compile_guide(self, outline):
        """Write all sections and compile the guide"""
        print("Writing guide sections and compiling...")
        completed_sections = []

        # Process sections one by one to maintain context flow
        for section in outline.sections:
            print(f"Processing section: {section.title}")

            # Build context from previous sections
            previous_sections_text = ""
            if completed_sections:
                previous_sections_text = "# Previously Written Sections\n\n"
                for title in completed_sections:
                    previous_sections_text += f"## {title}\n\n"
                    previous_sections_text += self.state.sections_content.get(title, "") + "\n\n"
            else:
                previous_sections_text = "No previous sections written yet."

            # Run the content crew for this section
            result = content_crew.crew().kickoff(inputs={
                "section_title": section.title,
                "section_description": section.description,
                "audience_level": self.state.audience_level,
                "previous_sections": previous_sections_text,
                "draft_content": ""
            })

            # Store the content
            self.state.sections_content[section.title] = result.raw
            completed_sections.append(section.title)
            print(f"Section completed: {section.title}")

        # Compile the final guide
        guide_content = f"# {outline.title}\n\n"
        guide_content += f"## Introduction\n\n{outline.introduction}\n\n"

        # Add each section in order
        for section in outline.sections:
            section_content = self.state.sections_content.get(section.title, "")
            guide_content += f"\n\n{section_content}\n\n"

        # Add conclusion
        guide_content += f"## Conclusion\n\n{outline.conclusion}\n\n"

        # Save the guide
        with open("output/complete_guide.md", "w") as f:
            f.write(guide_content)

        print("\nComplete guide compiled and saved to output/complete_guide.md")
        return "Guide creation completed successfully"

### Run the flow and plot it

In [9]:
"""Generate a visualization of the flow"""
my_flow = GuideCreatorFlow()
my_flow.plot("guide_creator_flow")
print("Flow visualization saved to guide_creator_flow.html")

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: GuideCreatorFlow                                                                                         │
│  ID: badcef52-126d-40ae-82ac-5de560f30ea0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Plot saved as guide_creator_flow.html
Flow visualization saved to guide_creator_flow.html


In [10]:
# Run the guide creator flow

result = await my_flow.kickoff_async()
print("\n=== Flow Complete ===")
print("Your comprehensive guide is ready in the output directory.")
print("Open output/complete_guide.md to view it.")

 Flow started with ID: badcef52-126d-40ae-82ac-5de560f30ea0

=== Create Your Comprehensive Guide ===

Creating a guide on Mazda Miata exhaust systems for intermediate audience...

Creating guide outline...

Error executing listener create_guide_outline: The model openrouter/openai/gpt-5-mini does not support 
response_format for provider 'openrouter'. Please remove response_format or use a supported model.

ValueError: The model openrouter/openai/gpt-5-mini does not support response_format for provider 'openrouter'. Please remove response_format or use a supported model.